## Instalamos las bibliotecas necesarias  
 Instala las bibliotecas principales (langchain) y sus integraciones con diversos modelos de lenguaje (OpenAI, Groq, Google Gemini).  

 La opción -qU asegura una instalación silenciosa y que tengamos las últimas versiones disponibles.

In [ ]:
!pip install langchain -qU

!pip install langchain-openai -qU
!pip install langchain-groq -qU
!pip install langchain-google-genai -qU

## Importación de las clases necesarias

Este bloque de código importa las clases necesarias de langchain para crear y utilizar los modelos de lenguaje de OpenAI, Groq y Google.  

Estas clases son un envoltorio (*wrapper*) para las APIs de sus respectivos propietarios (OpenAI, Groq y Google)

Tambien las clases que proporciona Langchain para la creación de mensajes de usuario y sistema.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.messages import HumanMessage, SystemMessage

## Carga Segura de Claves API

Utilizamos la funcionalidad de Google Colab para cargar de forma segura las claves API necesarias para acceder a los modelos de lenguaje. Las claves API de OpenAI, Groq, y Google se obtienen del sistema de configuración de Colab y se asignan a variables para su posterior uso. Este proceso evita la necesidad de codificar las claves directamente en el código, lo que representa una mejor práctica de seguridad.

In [ ]:
from google.colab import userdata

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# LANGCHAIN_TRACING_V2=True
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY="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"
# LANGCHAIN_PROJECT="pr-minty-idiom-15"

In [ ]:
modelo1 = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY)
modelo2 = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)
modelo3 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", api_key=GOOGLE_API_KEY)

mensajes = [
            SystemMessage("Proporciona una frase hecha (idiom) en inglés, equivalentes a la que te proporcione. Responde con un frase y solo con la frase."),
            HumanMessage("Estar en la edad del pavo")
           ]



In [ ]:
modelo1.invoke(mensajes)

In [ ]:
modelo2.invoke(mensajes)

In [ ]:
modelo3.invoke(mensajes)

### LangChain también admite entradas de modelos de chat mediante cadenas o formato OpenAI  
Son equivalentes los siguientes:

In [ ]:
modelo2.invoke("Traduce a un frase equivalente (idiom) en inglés: Más vale pájaro en mano que ciento volando")

In [ ]:
modelo2.invoke([
    {"role":"system", "content": "Proporciona una frase hecha (idiom) en inglés, equivalentes a la que te proporcione. Responde con un frase y solo con la frase."},
    {"role":"user", "content": "Más vale pájaro en mano que ciento volando"}
    ])

Observa que la respuesta del metodo .invoque() devuelve mucha informacion util o que podria ser util en aplicaciones mas complejas.  

AIMessage es una clase que contiene información sobre la respuesta de un modelo de lenguaje.

Representa el resultado del modelo y consta tanto del resultado sin procesar como devuelto por el modelo junto con campos estandarizados (por ejemplo, llamadas a herramientas, metadatos de uso) agregados por el marco LangChain.

- content (str): El contenido textual de la respuesta del modelo.

- additional_kwargs (dict): Un diccionario para información adicional específica del proveedor del modelo (puede estar vacío en algunos casos).

- response_metadata (dict): Información sobre la respuesta, como el uso de tokens, el nombre del modelo, etc.

- id (str): Un identificador único para el mensaje.

- usage_metadata (dict): Información de uso de tokens (entrada, salida y total).

In [ ]:
ai_message = modelo2.invoke([
    SystemMessage("Proporciona una frase hecha (idiom) en inglés, equivalentes a la que te proporcione. Responde con un frase y solo con la frase."),
    HumanMessage("Tener la mosca detrás de la oreja")
            ])

print("Contenido de la respuesta (content):\n", ai_message.content)
print("========")
print("Metadatos de la respuesta (response_metadata): \n", ai_message.response_metadata)
print("========")
print("Tokens utilizados (usage_metadata): \n", ai_message.usage_metadata)
print("========")
print("Identificador de la respuesta (id):", ai_message.id)
print("========")
print("Uso de tokens detallado(response_metadata['token_usage']:\n", ai_message.response_metadata['token_usage'])

## Streaming

**Conceptos Clave:**

- **Modelos de Chat "Ejecutables":**
    
    - Imagina que los modelos de chat (como GPT, Gemini, etc.) son como funciones que puedes "ejecutar" o "llamar".
        
    - En Langchain, estos modelos tienen una interfaz estándar, lo que significa que todos se comportan de manera similar cuando los usas.
        
    
- **Modos de Invocación:**
    
    - **Invocación Normal (Síncrona):** Es como preguntar algo y esperar a que el modelo te dé la respuesta completa de una sola vez.
        
    - **Invocación Asíncrona:** Es como preguntar algo y dejar que el modelo trabaje en segundo plano. Puedes hacer otras cosas mientras esperas la respuesta.
        
    - **Transmisión (Streaming):** Es como preguntar algo y recibir la respuesta poco a poco, palabra por palabra o token por token, en lugar de esperar a que esté completa.
        
    
- **Transmisión de Tokens:**
    
    - Los modelos de lenguaje generan texto dividiéndolo en "tokens". Un token puede ser una palabra, una parte de una palabra o un signo de puntuación.
        
    - La transmisión de tokens significa que el modelo te envía cada token a medida que lo genera, en lugar de esperar a tener toda la respuesta.
        
    

**¿Por qué es Útil la Transmisión de Tokens?**

- **Experiencia de Usuario Mejorada:**
    
    - En lugar de esperar un largo tiempo sin saber si el modelo está funcionando, el usuario ve la respuesta aparecer gradualmente. Esto hace que la interacción sea más fluida y natural.
        
    - Es como ver a alguien escribir en tiempo real en lugar de esperar a que termine de escribir todo el texto.
        
    
- **Respuestas Más Rápidas (Aparentemente):**
    
    - Aunque el tiempo total para generar la respuesta puede ser el mismo, la transmisión hace que parezca que la respuesta llega más rápido, ya que el usuario ve el texto aparecer poco a poco.
        
    
- **Procesamiento en Tiempo Real:**
    
    - Puedes empezar a procesar la respuesta del modelo a medida que se genera, sin tener que esperar a que esté completa. Esto es útil en aplicaciones que necesitan reaccionar a la respuesta del modelo en tiempo real.